In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('BBGM_League_93_all_seasons_Average_Stats.csv')

In [ ]:
df_lim = df[df.MP * df.G > 2000]

In [ ]:
sorted([(v,k) for k,v in df_lim.corr().loc['Ovr'].items()])[::-1]
cols = ['EWA','PER',"VORP","BPM","WS",'WS/48','Ovr']
df_lim[cols].corr().round(2)['Ovr']#.style.background_gradient()

In [ ]:
bins = pd.cut(df.MP*df.G,40)

In [ ]:
things = []
for b in bins.unique():
    corr = df[bins == b][['Ovr','PER']].corr()['Ovr']["PER"]
    if not np.isnan(corr):
        things.append([b.mid,corr,(bins==b).sum()])
things = np.array(things)

In [ ]:
def get_curve(x):
    scale = x[0]
    offset = x[1]
    slope = x[2]
    xv = things[:,0]
    yv = things[:,1]
    pred = scale * (np.tanh( (xv-offset)*slope )*0.5 + 0.5)
    #print(x,pred)
    #print()
    return ( ((pred - yv)*(things[:,2])) **2) .mean()

In [ ]:
import scipy.optimize as opt
res = opt.minimize(get_curve,[0.8,1000,1/500])

In [ ]:
res.x
#np.tanh((things[:,0]-100)/200)

In [ ]:
scale = res.x[0]
offset =res.x[1]
slope =res.x[2]
plt.style.use('seaborn-white')
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
label = "{:.2f} * sigmoid( (min - {:.0f}) * {:.01e} )".format(*res.x)
plt.scatter(things[:,0],things[:,1],label='true')
plt.scatter(things[:,0],scale * (np.tanh( (things[:,0]-offset)*slope )*0.5 + 0.5),c='r',label=label)
plt.legend(frameon=True)
plt.xlabel('Minutes')
plt.ylabel('correlation')
plt.title("PER correlation")
plt.ylim(0,1)
plt.grid()
plt.tight_layout()

In [ ]:
1/1300

In [ ]:
import statsmodels.api as sm

In [ ]:
clf = sm.WLS(df.Ovr,sm.add_constant(df[['PER']]),500+df.MP*df.G)
res_clf = clf.fit()

In [ ]:
res_clf.summary()

In [ ]:
dfs = df.sample(frac=0.1)
plt.scatter(dfs.PER,dfs.Ovr,s=np.sqrt(dfs.MP*dfs.G),alpha=0.1)


plt.xlim(0,30)
plt.ylim(20,80)
plt.xlabel('PER')
plt.ylabel('Ovr')
plt.scatter(dfs.PER, 31.693+dfs.PER* 1.531,label='32 + per*1.5')
plt.scatter( dfs.PER,36+dfs.PER*1.25,label='36 + per*1.25')
plt.legend()